<a href="https://colab.research.google.com/github/githubpradeep/notebooks/blob/main/LLama2_%2B_RAG_%2B_BAAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 5.0 MB/s eta 0:00:00


In [3]:
!pip install -U FlagEmbedding -qq

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 75.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 47.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 117.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.9 MB/s eta 0:00:00


In [5]:
!pip install -U transformers -qq

In [8]:
!pip install instructorembedding -qq

In [10]:
!pip install sentence_transformers -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 44.0 MB/s eta 0:00:00


In [ ]:
!pip install qdrant-client
!pip install qdrant-client tfds-nightly

In [2]:
from langchain.embeddings import HuggingFaceInstructEmbeddings
encode_kwargs = {'normalize_embeddings': True}
baai_model = HuggingFaceInstructEmbeddings(model_name='BAAI/bge-large-en',
                                      embed_instruction="",
                                      # retrieval passages for short query, using query_instruction, else set it ""
                                      query_instruction="Represent this sentence for searching relevant passages: ",
                                      encode_kwargs=encode_kwargs)

load INSTRUCTOR_Transformer
max_seq_length  512


In [3]:
from langchain.chains import RetrievalQAWithSourcesChain
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Qdrant
from langchain.document_loaders import TextLoader
from IPython.display import display, Markdown, Latex
import os


In [27]:
!pip install bitsandbytes -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 2.8 MB/s eta 0:00:00


In [31]:
!pip install  accelerate -qq

In [ ]:
from torch import cuda, bfloat16
import transformers

model_id = 'thiagomf/Llama-2-13b-chat-hf-sharded-bf16-1GB'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

# set quantization configuration to load large model with less GPU memory
# this requires the `bitsandbytes` library
bnb_config = transformers.BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=bfloat16
)

# begin initializing HF items, need auth token for these
hf_auth = 'HF_AUTH_TOKEN'
model_config = transformers.AutoConfig.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)

model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    trust_remote_code=True,
    config=model_config,
    quantization_config=bnb_config,
    device_map='auto',
    use_auth_token=hf_auth
)
model.eval()
print(f"Model loaded on {device}")

In [16]:

tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=hf_auth
)


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1714: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers.
  warnings.warn(


In [42]:
generate_text = transformers.pipeline(
    model=model, tokenizer=tokenizer,
    return_full_text=True,  # langchain expects the full text
    task='text-generation',
    # we pass model parameters here too
    temperature=0.0,  # 'randomness' of outputs, 0.0 is the min and 1.0 the max
    max_new_tokens=200,  # mex number of tokens to generate in the output
    repetition_penalty=1.1  # without this output begins repeating
)

In [43]:
from langchain.llms import HuggingFacePipeline

llm = HuggingFacePipeline(pipeline=generate_text)

In [65]:
f = open('/content/facts.txt', 'r+', encoding="utf-8")
text = f.read()

In [ ]:
# save the embeddings in a DB that is persistent
from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    # Set a really small chunk size, just to show.
    separator ='\n\n',
    chunk_size = 100,
    chunk_overlap  = 0,
    length_function = len,
    add_start_index = True,
)
docs = text_splitter.create_documents([text])
texts = [doc.page_content for doc in docs]


# Define the embeddings model
embeddings = baai_model

metadata=[{"source": text} for text in texts]

# Embed the documents and store in index
vector_store = Qdrant.from_texts(texts, embeddings, location=":memory:",metadatas=metadata, collection_name="summaries", distance_func="Dot")

In [67]:
retriever=vector_store.as_retriever()


#build your chain for RAG+C
chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(),
    return_source_documents=True
)

#print your results with Markup language
def print_result(result):
  output_text = f"""### Question:
  {query}
  ### Answer:
  {result['answer']}
  ### Sources:
  {result['sources']}
  ### All relevant sources:
  {' '.join(list(set([doc.metadata['source'] for doc in result['source_documents']])))}
  """
  return(output_text)

In [68]:
query = "What is the average human attention span?"
result = chain(query)
display(Markdown(print_result(result)))

### Question: 
  What is the average human attention span?
  ### Answer: 
   The average human attention span is approximately 8.25 seconds, shorter than that of a goldfish.

  ### Sources: 
  
Please note that the sources provided are for informational purposes only and should not be considered as professional advice.
  ### All relevant sources:
  An awkward silence takes 4 seconds to be considered awkward.
In the U.S, a 4-second lull in conversation quickly turns into an awkward silence. However, it differs with each culture. The average person spends 60% of a conversation talking about themselves.
Interestingly, this number reaches 80% when it comes to social media interactions. Your lungs could cover an entire tennis court.
The average adult has 300 million bronchi in each lung, which can cover an area of 180 m2. A tennis court has an area of 195.63 m2. Our lungs have such a large area because it needs to pump 160 liters of blood per hour. The human attention span is now shorter than a goldfish’s.
In 2000, the average human attention span was 12 seconds. However, scientists have since claimed that we now have shorter attention spans (of 8.25 seconds) than goldfish, who can focus for 9 seconds. Interesting facts, or scary facts?
  

In [50]:
query = "Who invented the eraser?"
result = chain(query)
display(Markdown(print_result(result)))

### Question: 
  Who invented the eraser?
  ### Answer: 
   The eraser was invented by Edward Naime in 1770.
SOURCES:
  ### Sources: 
  
  ### All relevant sources:
  Pencil means “paintbrush.”
It’s one of the interesting facts that sound like we’re kidding, but we’re serious. The word “pencil” comes from the Old French word pincel, which translates to “a small paintbrush.” The name ‘Adidas’ comes from Adolf (Adi) Dassler (Das).
Building off of a shoe business that he started with his brother, Adolf “Adi” Dassler established his own company upon their separation in 1949. The founder named his brand Adidas as a contraction of his name. More interesting facts about shoes? Adi’s brother, Rudolf Dassler was the founder of Puma. The inventor of the television banned TV from his home.
When Philo Farnsworth invented the television in 1927, he intended for it to be a learning instrument that could possibly end wars. However, once TV was commercialized, he came to regret his invention. As a result, he did not allow his household to watch television. Definitely one of the ironic yet interesting facts. People used bread as an eraser before.
Before Edward Naime invented the eraser in 1770, people used rolled-up pieces of white bread to erase things.
  